In [1]:
import os

import pyarrow as pa

import pandas as pd
from src.fondant.pipeline import Pipeline

from src.fondant.pipeline import Pipeline
from src.fondant.pipeline.runner import DockerRunner
from src.fondant.pipeline.compiler import DockerCompiler

from src.fondant.component import PandasTransformComponent
os.environ["DOCKER_DEFAULT_PLATFORM"]="linux/amd64"

In [2]:

# generate some data
data = pd.DataFrame(
        {
            "x": [1, 2, 3],
            "y": [4, 5, 6],
        },
        index=pd.Index(["a", "b", "c"], name="id"),
    )

data.to_parquet("./foobar/sample.parquet")

In [3]:



pipeline = Pipeline(name="foo", description="bar", base_path="/Users/georgeslorre/ML6/internal/fondant-xmas/foobar")


dataset = pipeline.read(
    name_or_path="components/load_from_parquet",
    arguments={
        "dataset_uri": "../../foobar/sample.parquet",
    },
    produces={"x": pa.int32(), "y": pa.int32()},
)

DockerCompiler().compile(pipeline=pipeline)
DockerRunner().run(input="./docker-compose.yml")


[2023-12-28 16:42:27,974 | src.fondant.pipeline.compiler | INFO] Compiling foo to docker-compose.yml
[2023-12-28 16:42:27,975 | src.fondant.pipeline.compiler | INFO] Base path found on local system, setting up /Users/georgeslorre/ML6/internal/fondant-xmas/foobar as mount volume
[2023-12-28 16:42:27,977 | src.fondant.pipeline.pipeline | INFO] Sorting pipeline component graph topologically.
[2023-12-28 16:42:27,993 | src.fondant.pipeline.pipeline | INFO] All pipeline component specifications match.
[2023-12-28 16:42:27,994 | src.fondant.pipeline.compiler | INFO] Compiling service for load_from_parquet
[2023-12-28 16:42:27,995 | src.fondant.pipeline.compiler | INFO] Found Dockerfile for load_from_parquet, adding build step.
[2023-12-28 16:42:28,001 | src.fondant.pipeline.compiler | INFO] Successfully compiled to docker-compose.yml


Starting pipeline run...
#1 [load_from_parquet internal] load .dockerignore
#1 transferring context: 2B done
#1 DONE 0.0s

#2 [load_from_parquet internal] load build definition from Dockerfile
#2 transferring dockerfile: 650B done
#2 DONE 0.0s

#3 [load_from_parquet internal] load metadata for docker.io/library/python:3.8-slim
#3 ...

#4 [load_from_parquet auth] library/python:pull token for registry-1.docker.io
#4 DONE 0.0s

#3 [load_from_parquet internal] load metadata for docker.io/library/python:3.8-slim
#3 DONE 1.6s

#5 [load_from_parquet 1/7] FROM docker.io/library/python:3.8-slim@sha256:d1cba0f8754d097bd333b8f3d4c655f37c2ede9042d1e7db69561d9eae2eebfa
#5 DONE 0.0s

#6 [load_from_parquet internal] load build context
#6 transferring context: 89B done
#6 DONE 0.0s

#7 [load_from_parquet 6/7] WORKDIR /component/src
#7 CACHED

#8 [load_from_parquet 5/7] RUN pip3 install fondant[component,aws,azure,gcp]@git+https://github.com/ml6team/fondant@main
#8 CACHED

#9 [load_from_parquet 3/7] C

 Container foo-load_from_parquet-1  Recreate
 Container foo-load_from_parquet-1  Recreated
foo-load_from_parquet-1  | [2023-12-28 15:42:32,964 | fondant.cli | INFO] Component `LoadFromParquet` found in module main
foo-load_from_parquet-1  | [2023-12-28 15:42:32,975 | fondant.component.executor | INFO] Dask default local mode will be used for further executions.Our current supported options are limited to 'local' and 'default'.
foo-load_from_parquet-1  | [2023-12-28 15:42:32,977 | fondant.component.executor | INFO] No matching execution for component detected
foo-load_from_parquet-1  | [2023-12-28 15:42:32,977 | root | INFO] Executing component
foo-load_from_parquet-1  | [2023-12-28 15:42:32,977 | main | INFO] Loading dataset from the hub...
foo-load_from_parquet-1  | [2023-12-28 15:42:33,135 | main | INFO] Index column not specified, setting a globally unique index
foo-load_from_parquet-1  | [2023-12-28 15:42:33,181 | root | INFO] Creating write task for: /foobar/foo/foo-20231228164227

[                                        ] | 0% Completed | 680.46 us
[########################################] | 100% Completed | 106.71 ms
foo-load_from_parquet-1 exited with code 0
Finished pipeline run.


In [4]:
class Addition(PandasTransformComponent):
    def __init__(self, *_, **__):
        pass

    def transform(self, dataframe: pd.DataFrame) -> pd.DataFrame:
        dataframe['z'] = dataframe['x'] + dataframe['y']
        return dataframe


In [5]:
%tb
_ = dataset.execute(component=Addition, produces={"z": pa.int32()}, consumes={"x": pa.int32(), "y": pa.int32()})

No traceback available to show.
[2023-12-28 16:42:37,018 | fondant.component.executor | INFO] Dask default local mode will be used for further executions.Our current supported options are limited to 'local' and 'default'.
[2023-12-28 16:42:37,020 | fondant.component.executor | INFO] Previous component `load_from_parquet` is not cached. Invalidating cache for current and subsequent components
[2023-12-28 16:42:37,020 | fondant.component.executor | INFO] Caching disabled for the component
[2023-12-28 16:42:37,020 | root | INFO] Executing component


FileNotFoundError: An error occurred while calling the read_parquet method registered to the pandas backend.
Original Message: /foobar/foo/foo-20231228164227/load_from_parquet

In [8]:
import dask.dataframe as dd
df = dd.read_parquet("/foobar/foo/foo-20231228164227/load_from_parquet")